In [38]:
import pandas
df = pandas.read_csv("Volcanoes.txt")
df

,VOLCANX020,NUMBER,NAME,LOCATION,STATUS,ELEV,TYPE,TIMEFRAME,LAT,LON
0,509.0,1201-01=,Baker,US-Washington,Historical,3285.0,Stratovolcanoes,D3,48.776798,-121.810997
1,511.0,1201-02-,Glacier Peak,US-Washington,Tephrochronology,3213.0,Stratovolcano,D4,48.111801,-121.111000
2,513.0,1201-03-,Rainier,US-Washington,Dendrochronology,4392.0,Stratovolcano,D3,46.869801,-121.751000
3,515.0,1201-05-,St. Helens,US-Washington,Historical,2549.0,Stratovolcano,D1,46.199799,-122.181000
4,516.0,1201-04-,Adams,US-Washington,Tephrochronology,3742.0,Stratovolcano,D6,46.205799,-121.490997
...,...,...,...,...,...,...,...,...,...,...
57,580.0,1210-03-,Valles Caldera,US-New Mexico,Pleistocene-Fumarolic,3430.0,Caldera,Q,35.869999,-106.570999
58,581.0,1209-02-,Sunset Crater,US-Arizona,Dendrochronology,2447.0,Cinder cone,D6,35.369999,-111.501000
59,584.0,1203-20-,Amboy,US-California,Holocene,288.0,Cinder cone,U,34.549999,-115.780998
60,586.0,1210-02-,Zuni-Bandera,US-New Mexico,Anthropology,2550.0,Volcanic field,D7,34.799999,-108.001000


In [39]:
lat = list(df["LAT"])
long = list(df["LON"])
names = list(df["NAME"])
elev = list(df["ELEV"])

In [57]:
#Function to give markers colours based on elevations
#Call function when adding colour to markers
def color_generator(elevation):
    if elevation<1000:
        return "green"
    elif 1000<=elevation<3000:
        return "purple"
    else:
        return "red"

#FOR HELP -> dir(folium)
#ADDITIONAL HELP -> help(folium.characteristic) where characteristic is Marker, CircleMarker etc
import folium

#Allows HTML code to be inserted in the popup for each marker-> for formatting text type,font,adding urls etc
# html = """
# Volcano name:<br>
# <a href="https://www.google.com/search?q=%%22%s%%22" target="_blank">%s</a><br>
# Height: %s m
# """

#Create a map object with a base location, a specific zoom and type
mp = folium.Map(location=[40.57,-115.49], zoom_start=6, tiles = "Stamen Terrain")

#Instead of adding characteristics/layers to the map separately -> We add them to a FeatureGroup
#This allows us to organise the characteristics/layers more easily
#FeatureGroup for volcanoes
fgv = folium.FeatureGroup(name="Volcanoes")

#Add point layer for volcano locations to the feauture group
#Zip allows us to traverse the 4 lists simultaneously
for la,lo,na,el in zip(lat,long,names,elev):
    #The iframe function allows us to implement the html code into the map for the popup option
    #iframe = folium.IFrame(html=html % (na, na, el), width=200, height=100)
    #We then add a marker with specific data for location, text, colour etc
#     fg.add_child(folium.Marker(location=[la,lo], popup=folium.Popup(iframe), parse_html=True,
#                                icon=folium.Icon(color=color_generator(el))))
#     fg.add_child(folium.Marker(location=[la,lo], popup=folium.Popup(str(na)+" ("+str(el)+" mtrs)"),
#                                parse_html=True, icon=folium.Icon(color='red')))
    #Circle marker with characteristics
    fgv.add_child(folium.CircleMarker(location=[la,lo], popup=str(na)+" ("+str(el)+" mtrs)", radius=10,
                                     fill_color=color_generator(el), color='grey', fill_opacity=0.7))

#Feature group for mapping countries and population
fgp = folium.FeatureGroup(name="Population")
#Add polygon layer for mapping countries to the feature group
#It adds the data stored in the world.json file to the map
#The json file stored data like coordinates for the countries shape and the population etc
#Colour the countries based on their population using lambda function
fgp.add_child(folium.GeoJson(data=open("world.json", "r", encoding = "utf-8-sig").read(),
                           style_function = lambda x: {'fillColor':'green' if x['properties']['POP2005']
                           < 10000000 else 'orange' if 10000000 <= x['properties']['POP2005'] < 20000000
                           else 'red'}))

#We then add each FeatureGroup with the characteristics/layers to the map object thus building the final map
mp.add_child(fgv)
mp.add_child(fgp)
#Add layercontrol to the map object but only after adding the feature group
#Otherwise it wont have any layers to control
mp.add_child(folium.LayerControl())

mp.save("Map1.html")